In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
from scipy import stats

In [3]:
import re

In [4]:
data= pd.read_csv('mobile phone price prediction.csv')

In [5]:
data.head(1)

,Unnamed: 0,Name,Rating,Spec_score,No_of_sim,Ram,Battery,Display,Camera,External_Memory,Android_version,Price,company,Inbuilt_memory,fast_charging,Screen_resolution,Processor,Processor_name
0,0,Samsung Galaxy F14 5G,4.65,68,"Dual Sim, 3G, 4G, 5G, VoLTE,",4 GB RAM,6000 mAh Battery,6.6 inches,50 MP + 2 MP Dual Rear &amp; 13 MP Front Camera,"Memory Card Supported, upto 1 TB",13,"9,999",Samsung,128 GB inbuilt,25W Fast Charging,2408 x 1080 px Display with Water Drop Notch,Octa Core Processor,Exynos 1330


In [6]:
data.duplicated().sum()

0

In [7]:
data=data.drop(columns = ['Unnamed: 0'],axis='columns')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1370 entries, 0 to 1369
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               1370 non-null   object 
 1   Rating             1370 non-null   float64
 2   Spec_score         1370 non-null   int64  
 3   No_of_sim          1370 non-null   object 
 4   Ram                1370 non-null   object 
 5   Battery            1370 non-null   object 
 6   Display            1370 non-null   object 
 7   Camera             1370 non-null   object 
 8   External_Memory    1370 non-null   object 
 9   Android_version    927 non-null    object 
 10  Price              1370 non-null   object 
 11  company            1370 non-null   object 
 12  Inbuilt_memory     1351 non-null   object 
 13  fast_charging      1281 non-null   object 
 14  Screen_resolution  1368 non-null   object 
 15  Processor          1342 non-null   object 
 16  Processor_name     1370 

In [10]:
temp=data[data.Camera.str.contains('Display')]
data.insert(7,'Dual_display',temp['Camera'])
data.iloc[temp.index,[8]]=temp[['External_Memory']]

In [11]:
data.loc[1246,['Battery','Display','Camera','External_Memory',
               'fast_charging','Screen_resolution','Ram','Inbuilt_memory','Processor','Processor_name']]=['6000 mAh Battery','6.75 inches',
                                                      '48 MP + 5 MP + 2 MP Triple Rear &amp; 8 MP','Memory Card (Hybrid)',
                                                      '22.5W Fast Charging',' 720 x 1600 px Display','4 GB RAM','128 GB inbuilt',np.nan,np.nan]
data.loc[264,['Name','Rating','Spec_score','No_of_sim','Ram','Battery','Display','Camera','External_Memory','Price','company','Inbuilt_memory',
              'fast_charging','Screen_resolution','Processor','Processor_name',]]=['Realme 6i(6GB RAM + 64GB)',3.85,76,'Dual Sim, 3G, 4G, VoLTE',
                                                                                   '6GB RAM','4300 mAh Battery','6.5 inches',
                                                                                   '48 MP Quad Rear &amp; 16 MP Front Camera',
                                                                                   np.nan,'14,999','Realme','64 GB inbuilt','30W Fast Charging',
                                                                                   ' 1080 x 2400 px, 90 Hz Display','Octa Core','Helio G90T']
data.loc[264,'Dual_display']=np.nan

In [12]:
# External coloum
temp=data[data.External_Memory.notnull()]
ind=temp[temp.External_Memory.str.contains('MP')].index
data.iloc[ind,9]=np.nan
data.insert(10,'os','0')
temp=data[data.External_Memory.notnull()]
ind=temp[temp.External_Memory.str.contains('Android')][['External_Memory']]
data.iloc[ind.index,[10]]=ind
data.iloc[ind.index,9]=np.nan

temp=data[data.External_Memory.notnull()]
ind=temp[temp.External_Memory.str.contains('Harmony')][['External_Memory']]
data.iloc[ind.index,[10]]=ind
data.iloc[ind.index,9]=np.nan

temp=data[data.External_Memory.notnull()]
ind=temp[temp.External_Memory.str.contains('EMUI')][['External_Memory']]
data.iloc[ind.index,[10]]=ind
data.iloc[ind.index,9]=np.nan

data.External_Memory=data.External_Memory.str.replace('Memory Card Supported, upto','')
data.External_Memory=data.External_Memory.str.replace('Memory Card (Hybrid), upto','')
data.External_Memory=data.External_Memory.str.replace('Memory Card Not Supported','0 GB')
data.External_Memory=data.External_Memory.str.strip()

data.External_Memory.replace({'1 TB':'1024 GB','2 TB':'2048 GB','Memory Card (Hybrid)':'1024 GB',
                              'Memory Card Supported':'1024 GB','No FM Radio':'0 GB'},inplace=True)

In [14]:
data.Inbuilt_memory=data.Inbuilt_memory.replace(' 1 TB inbuilt','1024 GB inbuilt')
data.Inbuilt_memory=data.Inbuilt_memory.replace('2 TB inbuilt','2048 GB inbuilt')
data.Inbuilt_memory=data.Inbuilt_memory.str.replace('GB inbuilt','')

In [15]:
data.fast_charging=data.fast_charging.str.replace('W Fast Charging','')
data.fast_charging=data.fast_charging.str.replace('Fast Charging','15')
#for j in range(10000,150000,10000):
#  for i in D:
#    data.loc[data[(data.company==i) & (data['fast_charging'].isna()) & (data.Price<j)].index,['fast_charging']]=D[i]
#data['fast_charging'].fillna(15,inplace=True)

In [16]:
data.Screen_resolution=data.Screen_resolution.str.replace('px Display','')

In [17]:
#  change the data
data.Price=data.Price.str.replace(',','').astype(int)
data.Battery=data.Battery.str.replace('mAh Battery','').astype(int)
data.Rating=data.Rating.astype(float)
data.Spec_score=data.Spec_score.astype(int)
data.Display=data.Display.str.replace('inches','').astype(float)

In [19]:
# Ram coloum
temp=data[data.Ram.str.contains('inbuilt')]
data.iloc[temp.index,9]=temp.Ram
data.iloc[temp.index,4]=np.nan

data.loc[data[(data['Price'] <= 20000) & (data['Ram'].isnull())].index,'Ram']='6.0'
data.loc[data[(data['Price'] <= 50000) & (data['Ram'].isnull())].index,'Ram']='8.0'
data.loc[data[(data['Price'] <= 1500000) & (data['Ram'].isnull())].index,'Ram']='12.0'
data.Ram=data.Ram.str.replace('GB RAM','').astype(float)

In [21]:
#  change the data type
data.Ram=data.Ram.astype(float)
data.Battery=data.Battery.astype(int)
data.fast_charging=data.fast_charging.astype(float)

In [23]:
## No of sim
data.insert(4,'Dual Sim',data.No_of_sim.str.contains('Dual Sim').astype('int'))
data.insert(5,'VoLTE',data.No_of_sim.str.contains('VoLTE').astype('int'))
data.insert(6,'5G',data.No_of_sim.str.contains('5G').astype('int'))
data.insert(7,'Vo5G',data.No_of_sim.str.contains('Vo5G').astype('int'))
data.drop('No_of_sim',axis=1,inplace=True)

In [25]:
# foldable and dual
data.Dual_display.fillna('No',inplace=True)
data.insert(12,'Foldable Display',data.Dual_display.str.contains('Foldable').astype('int'))
data.insert(13,'Dual Display',data.Dual_display.str.contains('Dual').astype('int'))

In [27]:
# Androids version
data['OS']=data.os.apply(lambda x : str(x).split(' ')[0])
data['OS_v']=data.os.apply(lambda x : str(x).split(' ')[1] if len(str(x).split(' '))>1 else 0 ).str.replace('v','').fillna(0)
data['v']=data.Android_version.fillna('0').apply(lambda x : str(x).split(' ')[0] )
data.v=data.v.str.replace('7.1.1','7.1',)
data.Android_version=data.v.astype('float')+data.OS_v.astype('float')
data.drop(['OS','OS_v','v'],axis=1,inplace=True)

In [28]:
data['Water Drop Notch']=data.Screen_resolution.str.contains('Water Drop').astype('int')
data['with Punch Hole']=data.Screen_resolution.str.contains('Punch Hole').astype('int')
data.loc[933,'Screen_resolution']='1920 x 1080 '
data['H px']= data.Screen_resolution.apply(lambda s : re.findall(r'\d+',s)[0])
data['V px']=data.Screen_resolution.apply(lambda s : re.findall(r'\d+',s)[1])
data=data.drop('Screen_resolution',axis=1)

In [30]:
temp=data[data.Processor_name.notna()]
ind=temp[temp.Processor_name.str.contains('Core')].index
data.loc[ind,'Processor']=temp['Processor_name']

In [31]:

data.loc[1,['Processor_name']]= 'Snapdragon 450'
data.loc[2,['Processor_name']]= 'Exynos 850'
data.loc[744,['Processor_name']]= 'Exynos 850'
data.loc[16,['Processor_name']]= 'Exynos 1280'
data.loc[22,['Processor_name']]= 'Snapdragon 730'
data.loc[27,['Processor_name']]= 'Snapdragon 750G'
data.loc[203,['Processor_name']]= 'Snapdragon 8 Gen1 Plus'
data.loc[234,['Processor_name']]= 'Helio P22'
data.loc[288,['Processor_name']]= 'Dimensity 6100 Plus'
data.loc[297,['Processor_name']]= 'Snapdragon 8 Gen1'
data.loc[396,['Processor_name']]= 'Snapdragon 8 Gen2'
data.loc[510,['Processor_name']]= 'JLQ JR510'
data.loc[626,['Processor_name']]= 'Snapdragon 8 Gen2'
data.loc[682,['Processor_name']]= 'Snapdragon 7 Gen 3 Plus'
data.loc[720,['Processor_name']]= 'Helio G88'
data.loc[747,['Processor_name']]= 'Exynos 2100'
data.loc[750,['Processor_name']]= 'Exynos 2100'
data.loc[754,['Processor_name']]= 'Exynos 1280'
data.loc[756,['Processor_name']]= 'Snapdragon 782G'
data.loc[786,['Processor_name']]= 'Exynos 1280'
data.loc[897,['Processor_name']]= 'Dimensity 700'
data.loc[984,['Processor_name']]= 'Snapdragon 720G'
data.loc[1164,['Processor_name']]= 'Snapdragon 8 Gen 3'
data.loc[1225,['Processor_name']]= 'Dimensity 9000'
data.loc[1239,['Processor_name']]= 'Kirin 710A'
data.loc[1241,['Processor_name']]= 'Kirin 710F'
data.loc[1242,['Processor_name']]= 'Snapdragon 680'
data.loc[1248,['Processor_name']]= 'Snapdragon 680G'
data.loc[1276,['Processor_name']]= 'Kirin 9010'
data.loc[1277,['Processor_name']]= 'Kirin 9010'
data.loc[1282,['Processor_name']]= 'Kirin 9010'
data.loc[1283,['Processor_name']]= 'Kirin 9000'
data.loc[1291,['Processor_name']]= 'Kirin 9000'
data.loc[1234,['Processor_name']]= 'Kirin 710'
data.iloc[[655,689,692,1348,218,1246],21]=['other','unisoc SC9863','unisoc T606','Helio G25',' Helio G35','Kirin 710A']

In [32]:
data.company=data.company.str.lower()
data.Processor=data.Processor.str.lower()
data.Processor_name=data.Processor_name.str.lower()
data.Processor_name=data.Processor_name.str.strip()
data.Processor=data.Processor.str.replace('processor','').str.strip()

In [33]:
data.Processor_name=data.Processor_name.str.replace('google ','')
data.Processor_name=data.Processor_name.str.replace('samsung ','')
data.Processor_name=data.Processor_name.str.replace('qualcomm ','')
data.Processor_name=data.Processor_name.str.replace('sanpdragon','snapdragon')
data.Processor_name=data.Processor_name.str.strip()

In [34]:
data['Processor_']=data.Processor_name.apply(lambda x : str(x).split(' ')[0])

In [35]:
data['Processor_series']=data.Processor_name.apply(lambda x : str(x).split(' ')[1] if len(str(x).split(' '))>1 else 'no ditail')

In [36]:
data.Processor_.replace({'jlq':'other','quad':'other','tiger':'unisoc'},inplace=True)

In [37]:
def s(x):
    match=re.search(r'\d+', str(x))
    if match:
        return match.group()
data.Processor_series=data.Processor_series.apply(s)

In [38]:
data.insert(10,'Rear',data.Camera.apply(lambda x : str(x).split(';')[0]))
data.insert(11,'Front',data.Camera.apply(lambda x : str(x).split(';')[1] if len(str(x).split(';'))>1 else '0'))
data['No _of_Rear']=data.Rear.apply(lambda x: 4 if 'Quad' in x else 3 if 'Triple' in x else 2 if 'Dual' in x else 1)
data['No _of_Front']=data.Front.apply(lambda x: 4 if 'Quad' in x else 3 if 'Triple' in x else 2 if 'Dual' in x else 1)
data['Primary_rear_camera']=data.Rear.apply(lambda x : str(x).split(' ')[0])
data['Primary_front_camera']=data.Front.str.strip().apply(lambda x : str(x).split(' ')[0])

In [39]:
data.External_Memory=data.External_Memory.apply(s)

In [40]:
data['Number_of_core']=data.Processor.apply(lambda x: '8' if 'octa' in str(x) else '4' if 'quad' in str(x) else '9' if 'nine' in str(x) else '10')

In [41]:
data=data.drop(['Rear','Front','Camera','os','Processor_name','Processor','Dual_display'],axis=1)

In [42]:
data.Primary_front_camera=data.Primary_front_camera.astype('float')


In [44]:
data.to_csv('smart_phone.csv')